In [24]:
import random
import numpy as np
import pandas as pd

data = pd.read_csv("ga_input_0.1.csv")
features = data[["Latitude", "Longitude","Frequency"]].values
data1 = data["Cluster"].values
data2 = pd.read_csv("ga_input_0.2.csv")
data2 = data2["Cluster"].values
data3 = pd.read_csv("ga_input_0.3.csv")
data3 = data3["Cluster"].values
data4 = pd.read_csv("ga_input_0.4.csv")
data4 = data4["Cluster"].values
data5 = pd.read_csv("ga_input_0.5.csv")
data5 = data5["Cluster"].values
len(data1), len(data2), len(data3), len(data4), len(data5)

(149, 149, 149, 149, 149)

In [32]:


class ClusterGA:
    def __init__(self, n_clusters, population_size, max_iterations):
        self.n_clusters = n_clusters
        self.population_size = population_size
        self.max_iterations = max_iterations
        
    # def initialize_population(self, data):
    #     self.data = data
    #     self.population = []
    #     for i in range(self.population_size):
    #         clusters = [random.randint(0, self.n_clusters-1) for j in range(len(data))]
    #         self.population.append(clusters)
            
    #     print(len(data))
        # for cluster in self.population:
        #     print(cluster)
    def initialize_population(self, data):
        self.data = data
        self.population = []
        self.population.append([i for i in np.array(data1)])
        self.population.append([i for i in np.array(data2)])
        self.population.append([i for i in np.array(data3)])
        self.population.append([i for i in np.array(data4)])
        self.population.append([i for i in np.array(data5)])
        for cluster in self.population:
            print(cluster)
        
    def evaluate_fitness(self):
        fitness = []
        for individual in self.population:
            centroids = []
            cluster_frequencies = []
            for i in range(self.n_clusters):
                cluster_data = [self.data[j][:2] for j in range(len(self.data)) if individual[j] == i]
                cluster_freq = [self.data[j][2] for j in range(len(self.data)) if individual[j] == i]
                if len(cluster_data) > 0:
                    centroid = np.mean(cluster_data, axis=0)
                    variance = np.var(cluster_freq)
                else:
                    centroid = np.zeros(2)
                    variance = 0
                centroids.append(centroid)
                cluster_frequencies.append(variance)

            total_distance = 0
            for i in range(len(self.data)):
                cluster_index = individual[i]
                total_distance += np.linalg.norm(self.data[i][:2] - centroids[cluster_index])

            fitness.append(np.sum(cluster_frequencies) / total_distance)

        return fitness

        
    def select_parents(self, fitness):
        max_fitness = max(fitness)
        fitness = [max_fitness - f for f in fitness]
        total_fitness = sum(fitness)
        selection_prob = [f/total_fitness for f in fitness]
        
        parents = []
        for i in range(2):
            r = random.random()
            cum_prob = 0
            for j in range(len(self.population)):
                cum_prob += selection_prob[j]
                if cum_prob >= r:
                    parents.append(self.population[j])
                    break
                    
        return parents
        
    def crossover(self, parent1, parent2):
        crossover_point = random.randint(1, len(self.data)-2)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2
        
    def mutate(self, individual, mutation_rate):
        for i in range(len(individual)):
            if random.random() < mutation_rate:
                individual[i] = random.randint(0, self.n_clusters-1)
        return individual
        
    def run(self, data):
        self.initialize_population(data)
        best_fitness = float('inf')
        
        for i in range(self.max_iterations):
            fitness = self.evaluate_fitness()
            parents = self.select_parents(fitness)
            child1, child2 = self.crossover(parents[0], parents[1])
            child1 = self.mutate(child1, 0.01)
            child2 = self.mutate(child2, 0.01)
            self.population = [parents[0], parents[1], child1, child2] + [self.mutate(parent, 0.01) for parent in self.population[2:]]
            fitness = self.evaluate_fitness()
            best_index = np.argmin(fitness)
            best_clusters = self.population[best_index]
            if fitness[best_index] < best_fitness:
                best_fitness = fitness[best_index]
                best_clusters = self.population[best_index]
            
            print("Generation {}: Best fitness = {}, Best clusters = {}".format(i+1, best_fitness, best_clusters))
            
        
        return best_clusters

# data = np.array([
#     [19.1197 ,72.8464,10], #Andheri
#  [19.0544 ,72.8402,100],#bandra
#  [19.1645, 72.8493,3],#goregaon
#  [19.0669, 72.8774,7],#bkc
#  [19.076 , 72.8846,3],#kurla
#  [19.127 , 72.8291,11],#dnnagar
# ])

data = np.array(features)
ga = ClusterGA(n_clusters=15, population_size=5, max_iterations=100)

best_clusters = ga.run(data)

print("Best clusters found:", best_clusters)

[13, 0, 12, 11, 8, 12, 1, 13, 9, 6, 11, 3, 8, 1, 8, 1, 13, 7, 11, 4, 11, 13, 9, 9, 0, 2, 12, 13, 1, 3, 11, 12, 13, 2, 11, 1, 0, 8, 9, 2, 6, 8, 8, 11, 2, 11, 9, 1, 9, 0, 4, 9, 1, 8, 1, 13, 7, 0, 10, 11, 13, 9, 2, 9, 0, 12, 0, 12, 1, 3, 2, 13, 9, 0, 11, 0, 8, 3, 4, 7, 12, 4, 8, 11, 4, 0, 8, 3, 2, 13, 6, 8, 5, 14, 8, 12, 1, 9, 9, 1, 12, 0, 9, 3, 13, 0, 6, 7, 12, 11, 9, 0, 5, 6, 6, 1, 3, 8, 0, 0, 4, 12, 8, 5, 2, 0, 9, 11, 7, 2, 3, 11, 12, 1, 12, 11, 3, 6, 13, 12, 1, 7, 0, 6, 9, 1, 0, 13, 6]
[13, 0, 12, 13, 8, 12, 1, 13, 9, 6, 11, 3, 8, 1, 8, 1, 13, 7, 11, 4, 11, 13, 9, 9, 9, 2, 12, 13, 1, 3, 11, 12, 13, 2, 11, 1, 0, 8, 9, 2, 6, 8, 11, 11, 2, 11, 9, 1, 9, 0, 4, 9, 1, 11, 1, 13, 7, 0, 10, 11, 13, 9, 2, 9, 0, 12, 0, 13, 1, 3, 2, 13, 9, 0, 11, 0, 8, 3, 4, 7, 12, 4, 8, 11, 4, 0, 8, 3, 2, 13, 6, 8, 5, 14, 8, 12, 1, 9, 9, 1, 12, 0, 9, 3, 13, 0, 6, 7, 12, 11, 9, 0, 5, 5, 6, 1, 3, 8, 0, 0, 4, 12, 8, 5, 2, 0, 9, 11, 7, 3, 3, 11, 12, 1, 12, 11, 3, 6, 13, 12, 7, 7, 0, 6, 9, 1, 0, 13, 6]
[13, 0, 12, 11

In [33]:
print(best_clusters)

[10, 0, 9, 0, 8, 10, 5, 9, 0, 14, 4, 8, 6, 11, 1, 1, 13, 7, 11, 5, 7, 2, 9, 1, 0, 3, 12, 13, 0, 6, 11, 11, 14, 13, 11, 1, 0, 8, 9, 6, 6, 8, 9, 11, 2, 11, 9, 1, 9, 4, 4, 4, 7, 13, 6, 12, 7, 8, 8, 11, 13, 14, 0, 9, 0, 3, 0, 13, 5, 9, 2, 2, 14, 3, 3, 2, 13, 3, 3, 13, 13, 1, 14, 11, 14, 0, 4, 2, 2, 11, 6, 0, 7, 7, 6, 12, 2, 9, 9, 3, 1, 0, 4, 10, 13, 0, 3, 3, 12, 11, 13, 5, 3, 0, 6, 13, 2, 8, 5, 0, 9, 12, 1, 5, 10, 10, 11, 11, 9, 2, 9, 11, 7, 1, 12, 11, 5, 11, 11, 10, 5, 2, 0, 12, 9, 13, 2, 8, 6]


In [35]:
df = pd.read_csv('ga_output_0.2.csv')

In [36]:
df['GeneticCluster'] = best_clusters

In [37]:
df.head()

,Location,Latitude,Longitude,Kmeans,Frequency,Genetic,Genetic.1,GeneticFinal,GeneticCluster
0,Andheri East,19.115491,72.854202,3,62,3,4,21,10
1,Bandra West,19.058336,72.830267,3,82,4,3,3,0
2,Chembur East,19.055292,72.903055,0,48,3,1,0,9
3,Dadar West,19.019282,72.842875,1,35,0,4,0,0
4,Fort,18.933266,72.836019,1,16,1,2,13,8


In [38]:
df = df.rename(columns={'Cluster': 'Kmeans'})
df = df.rename(columns={'GeneticCluster': 'Genetic'})

In [39]:
df.head()

,Location,Latitude,Longitude,Kmeans,Frequency,Genetic,Genetic.1,GeneticFinal,Genetic
0,Andheri East,19.115491,72.854202,3,62,3,4,21,10
1,Bandra West,19.058336,72.830267,3,82,4,3,3,0
2,Chembur East,19.055292,72.903055,0,48,3,1,0,9
3,Dadar West,19.019282,72.842875,1,35,0,4,0,0
4,Fort,18.933266,72.836019,1,16,1,2,13,8


In [41]:
df.to_csv('ga_output_0.2.csv', index=False)

In [42]:
df2 = pd.read_csv('ga_output_0.2.csv')

# group the rows by the cluster label
grouped = df2.groupby('Genetic')

# create an empty list to store the centroids and frequency of each cluster
centroids = []
frequencies = []

# loop through each cluster label and calculate the centroid and frequency
for label, group in grouped:
    # calculate the centroid of the locations in the cluster
    centroid = group[['Latitude', 'Longitude']].mean().tolist()
    # calculate the total frequency of the locations in the cluster
    frequency = group['Frequency'].sum()
    # add the centroid and frequency to the lists
    centroids.append(centroid)
    frequencies.append(frequency)

# create a new DataFrame with the cluster name, centroid, and frequency
cluster_df = pd.DataFrame({'ClusterName': [f'Cluster {i}' for i in range(len(centroids))],
                           'Latitude': [c[0] for c in centroids],
                           'Longitude': [c[1] for c in centroids],
                           'Frequency': frequencies})

# write the new DataFrame to a CSV file
cluster_df.to_csv('cluster_output.csv', index=False)

In [49]:
df3 = pd.read_csv("cluster_output.csv")

# Calculate the frequency percentages
max_freq = df3["Frequency"].astype(int).max()
print(max_freq)

df3["Freq_Percent"] = df3["Frequency"] / max_freq
print(df3)

# Assign colors based on frequency percentages
df3["Color"] = ""
df3.loc[df3["Freq_Percent"] >= 0.75, "Color"] = "red"
df3.loc[(df3["Freq_Percent"] >= 0.5) & (df3["Freq_Percent"] < 0.75), "Color"] = "orange"
df3.loc[(df3["Freq_Percent"] >= 0.33) & (df3["Freq_Percent"] < 0.5), "Color"] = "yellow"
df3.loc[df3["Freq_Percent"] < 0.33, "Color"] = "green"

# Write the updated DataFrame to a new CSV file
df3.to_csv("cluster_output_color.csv", index=False)

750
   ClusterName   Latitude  Longitude  Frequency  Freq_Percent
0    Cluster_0  19.048099  72.849671        540      0.720000
1    Cluster_1  19.060669  72.895230        359      0.478667
2    Cluster_2  19.064766  72.856308        470      0.626667
3    Cluster_3  19.113852  72.912186        295      0.393333
4    Cluster_4  19.045321  72.870837        181      0.241333
5    Cluster_5  19.125974  72.926492        360      0.480000
6    Cluster_6  19.116584  72.854271        252      0.336000
7    Cluster_7  19.112685  72.938473        234      0.312000
8    Cluster_8  19.057521  72.830748        166      0.221333
9    Cluster_9  19.118646  72.868164        646      0.861333
10  Cluster_10  19.079521  72.863431        331      0.441333
11  Cluster_11  19.048473  72.843446        750      1.000000
12  Cluster_12  19.093515  72.884105        284      0.378667
13  Cluster_13  19.071749  72.873684        442      0.589333
14  Cluster_14  19.090913  72.867557        118      0.157333
